### CODING SAMPLE ###

**USE-CASE**
> The project seeks to build a webpage for every supreme court case argued since 1709. Here, I designed a database schema to best meet the needs of this project and implemented my design using Python and SQL. 

**TASKS**
- Scrapes data files from http://scdb.wustl.edu/data.php
- Preprocesses data - cleans and denormalizes dataframes
- Creates data tables according to database design schema
- Loads data into SQL database (to protect sensitive information, loading code is provided without database credentials)
- Writes data tables to csv 

**OUTPUT**

Three csv files of clean data tables ready for production use: 
- cases.csv
- terms.csv
- justices.csv

In [ ]:
# IMPORTS #
import pandas as pd

# Scraper Imports #  
import requests 
from bs4 import BeautifulSoup
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# SQL Imports # 
from sqlalchemy import create_engine

#### Scraping ####

In [ ]:
# UNZIPPER #
# @params scraped link to zipped csv
# @returns unzipped csv from scraped link
def link_to_csv(link):
    
    resp = urlopen(link)                                                        # open url
    zipfile = ZipFile(BytesIO(resp.read()))                                     # read zipfile
    fname = zipfile.namelist()[0]                                               # get the csv file name
    df = pd.read_csv(zipfile.open(fname), dtype=object, encoding='latin1')      # convert to pandas dateframe
    zipfile.close()                                                             # close zipfile we don't need
    
    return df

In [ ]:
# SCRAPER METHOD # 
# @params case_centered - boolean for whether to pull case or justice-centered data 
# @returns links to zipped csvs 
def scrape(case_centered):
    # Making a GET request from Washington University SCDB website
    scdb = requests.get('http://scdb.wustl.edu/data.php') 
    
    # check status code for response received 
    # success code - 200 
    # print(scdb) 
    
    # Parsing the HTML 
    soup = BeautifulSoup(scdb.content, 'html.parser') 

    links = []

    if case_centered:
        # Find by ID -- Case-Centered by Issue/LegalProvision
        case_centered = soup.find('div', id='SCDB_2022_01_case_LegalProvision')     # Identifies links using site-specific tag
        for link in case_centered.find_all('a'):                                    # Finds all links within tag
            links.append("http://scdb.wustl.edu/" + link.get('href'))
        raw_df = link_to_csv(links[0])                                              # Scrapes first link into csv
    else:
        # Find by ID -- Justice-Centered by Issue/LegalProvision
        justice_centered = soup.find('div', id='SCDB_2022_01_justice_LegalProvision')   # Identifies links using site-specific tag
        for link in justice_centered.find_all('a'):                                     # Finds all links within tag
            links.append("http://scdb.wustl.edu/" + link.get('href'))                  
        raw_df = link_to_csv(links[0])                                                  # Scrapes first link into csv
    
    return raw_df

#### Preprocessing ####

In [ ]:
# CASES - KEYS
# caseOriginKey
caseOriginKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - caseOriginKey.csv')
# petitionerkey
petitionerKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - petitionerKey.csv')
# respondentKey
respondentKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - respondentKey.csv')

# TERMS - KEYS
# issueKey
issueKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - issueKey.csv')

# JUSTICES - KEYS
# majOpinWriterKey
majOpinWriterKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - majOpinionWriterKey.csv')
#justiceNameKey
justiceNameKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - justiceNameKey.csv')

In [ ]:
# KEYS TO MANUALLY DECODE 

# jurisdictionKey
jurisdictionKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - jurisdictionKey.csv')

# caseDispositionKey
caseDispositionKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - caseDispositionKey.csv')

# decisionTypeKey
decisionTypeKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - decisionTypeKey.csv')

# decisionDirectionKey
decisionDirectionKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - decisionDirectionKey.csv')

# issueAreaKey
issueAreaKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - issueAreaKey.csv')

# voteKey
voteKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - voteKey.csv')

# justiceDirectionKey
justiceDirectionKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - justiceDirectionKey.csv')

# opinionKey
opinionKey = pd.read_csv('/Users/rhitommishra/Desktop/ballotpedia/scdb_keys/SCDB_keys - opinionKey.csv')


In [ ]:
# VARIABLES FROM SANDBOX #
sandbox_cases = ["caseId", "usCite", "caseName","caseOrigin", "caseDisposition",
                "dateArgument", "dateDecision", "petitioner", "respondent", 
                "jurisdiction", "decisionType", "decisionDirection", "majVotes", 
                "minVotes", "majOpinWriter", "issue", "issueArea"]

sandbox_justices = ["caseId", "caseName", "dateDecision", "opinion", 
                    "term", "justiceName", "vote", "direction"]

sandbox_terms = ["caseId", "chief", "term"]

In [ ]:
# CALCULATES PERCENTAGE OF MISSING DATA #
# @params raw (unprocessed) dataframe 
# @returns columns where more than 30% of data is missing
def missing(df):
    # Counts how many values are missing per column
    missing_count = df.isna().sum(axis=0)
    missing_cols = list(df.isna())

    # Counts how many null values total per column (again)
    value_count = len(df)

    # Makes dataframe of % missing and total missing values
    missing_percentage = round((missing_count/ value_count) * 100, 1)
    missing_df = pd.DataFrame({'columns': missing_cols, 'percentage': missing_percentage})

    # Drops all rows where there are no null values
    missing_df = missing_df[(missing_df['percentage']>=30)]

    return missing_df['columns']

In [ ]:
# DROPS COLUMNS WITH NULL OR MISSING DATA # 
# @params raw_df - unprocessed dataframe
#         sandbox_columns - list of columns required by webpage
# @returns clean datagrame
def clean(raw_df, sandbox_columns):
    # if in missing_df, drop from raw
    is_Missing = missing(raw_df)
    if is_Missing.empty:
        print("Error message!")
    else:
        df = raw_df.drop(is_Missing, axis = 1)
    
    # select required columns
    df_clean = df.loc[:, sandbox_columns]

    return df_clean

In [ ]:
# MAKES DICTIONARY FOR DECODING #
# @params df - dataframe to decode 
#         key - string column name of encoded normalizations
#         val - strong column name of decoded values
# @returns dictionary where encoded key references decoded value
def makeDict(df, key, val):
    keys = df[key].to_list()
    values = df[val].to_list()

    dfDict = {keys[i]: values[i] for i in range(len(keys))}
    return dfDict

In [ ]:
# DECODES NORMALIZED VARIABLES #
# @params df - dataframe to decode
#         df_key - dataframe of encoded variables
#         colName - column in dataframe to decode
# @returns dataframe with decoded column

def decode(df, df_key, colName):
    keys = df_key[colName + "Key"].to_list()
    values = df_key[colName + "Val"].to_list()
    keyDict = {str(keys[i]): values[i] for i in range(len(keys))}
    
    # print(keyDict)
    # print(keyDict['1'])
    decoded = df.replace({colName: keyDict})
    return decoded

#### Loading Data ####

In [ ]:
# LOADS DATAFRAME INTO SQL DATA TABLE #
# @params df - dataframe to load into data table
#         username, password, host, database - login info 

def load(df, username, password, host, database):
    # creates engine that connects to SQL database
    engine=create_engine("postgresql+psycopg2://" + username + ":"+ password + "@" + host+ ":5432/" + database)

    try:
        # uses to_sql function to load data into existant data tables
        df.to_sql('scotus_cases', engine, schema='public', if_exists='append', index=False)
    except Exception as error: ## try getting to exception ## 
        print("an error has occured: ", error)

    finally:
        # disconnects from server
        engine.dispose()
        print("Success!")
    

In [ ]:
def main():
    # Set up login info # 
    # Sample # 
    username = "xxx"
    password = "xxx"
    host = "xxx"
    database = "xxx"

    # Scrape raw csvs # 
    cases_raw = scrape(case_centered = True)
    justices_raw = scrape(case_centered = False)

    # Clean data #
    cases = clean(cases_raw, sandbox_cases)
    justices = clean(justices_raw, sandbox_justices)
    terms = clean(justices_raw, sandbox_terms)
    
    # Decode cases data #
    cases = decode(cases, jurisdictionKey, "jurisdiction")
    cases = decode(cases, caseDispositionKey, "caseDisposition")
    cases = decode(cases, decisionTypeKey, "decisionType")
    cases = decode(cases, decisionDirectionKey, "decisionDirection")
    cases = decode(cases, issueKey, "issue")
    cases = decode(cases, issueAreaKey, "issueArea")

    # Decode justices #
    justices = decode(justices, voteKey, "vote")
    justices = decode(justices, justiceDirectionKey, "direction")
    justices = decode(justices, opinionKey, "opinion")

    # Loading three main data tables # 
    load(cases, username, password, host, database)
    load(terms, username, password, host, database)
    load(justices, username, password, host, database)

    # Loading key tables #
    load(caseOriginKey, username, password, host, database)
    load(petitionerKey, username, password, host, database)
    load(respondentKey, username, password, host, database)
    load(decisionDirectionKey, username, password, host, database)
    load(issueKey, username, password, host, database)
    load(majOpinWriterKey, username, password, host, database)
    load(justiceNameKey, username, password, host, database)

    # Exporting into CSVs
    cases.to_csv('scdb_cases.csv', sep=',', index=False, encoding='utf-8')
    terms.to_csv('scdb_outcomes.csv', sep=',', index=False, encoding='utf-8')
    justices.to_csv('scdb_justices.csv', sep=',', index=False, encoding='utf-8')



In [ ]:
main()